In [1]:
!nvidia-smi

Tue Dec 29 13:45:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Wrapper

In [2]:
import numpy as np
import os
#os.environ.setdefault('PATH', '')
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)

class TimeLimit(gym.Wrapper):
    def __init__(self, env, max_episode_steps=None):
        super(TimeLimit, self).__init__(env)
        self._max_episode_steps = max_episode_steps
        self._elapsed_steps = 0

    def step(self, ac):
        observation, reward, done, info = self.env.step(ac)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self._max_episode_steps:
            done = True
            info['TimeLimit.truncated'] = True
        return observation, reward, done, info

    def reset(self, **kwargs):
        self._elapsed_steps = 0
        return self.env.reset(**kwargs)
    
class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
            #self.was_real_done = done # modified 2020.12.22 
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True, dict_space_key=None):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.

        If the environment uses dictionary observations, `dict_space_key` can be specified which indicates which
        observation should be warped.
        """
        super().__init__(env)
        self._width = width
        self._height = height
        self._grayscale = grayscale
        self._key = dict_space_key
        if self._grayscale:
            num_colors = 1
        else:
            num_colors = 3

        new_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(self._height, self._width, num_colors),
            dtype=np.uint8,
        )
        if self._key is None:
            original_space = self.observation_space
            self.observation_space = new_space
        else:
            original_space = self.observation_space.spaces[self._key]
            self.observation_space.spaces[self._key] = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3

    def observation(self, obs):
        if self._key is None:
            frame = obs
        else:
            frame = obs[self._key]

        if self._grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self._width, self._height), interpolation=cv2.INTER_AREA
        )
        if self._grayscale:
            frame = np.expand_dims(frame, -1)

        if self._key is None:
            obs = frame
        else:
            obs = obs.copy()
            obs[self._key] = frame
        return obs


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.

        Returns lazy array, which is much more memory efficient.

        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.

        This object should only be converted to numpy array before being passed to the model.

        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-1)
            self._frames = None
        return self._out

    def concatenate(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return self._force().shape[2]

    def __getitem__(self, i):
        return self._force()[:,:,i]

    def count(self):
        frames = self._force()
        return frames.shape[frames.ndim - 1]

    def frame(self, i):
        return self._force()[..., i]

def make_atari(env_id, max_episode_steps=None):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    if max_episode_steps is not None:
        print("what?")
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

def make_env(env_name):
    env = make_atari(env_name)
    env = wrap_deepmind(env, frame_stack=True)
    return env

## Util

In [3]:
from collections import deque
import numpy as np

import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Args

In [4]:
FRAME_END = 3e5

GAMMA  = 0.99

EPSILON      = 1.0
EPSILON_END  = 0.02
EPSILON_DECAY = 0.99998

LEARNING_RATE = 0.0001

BATCH_SIZE  = 32

MEMORY_SIZE = 10_000
MEMORY_SAMPLE_START = 1_000

MODEL_UPDATE_STEP =  1_000

RENDER = False
GIF_MAKE = False

ENV_NAME = "PongNoFrameskip-v4"

### DuelingDQN

In [5]:
class DDQN(nn.Module):
    def __init__(self,IN_CHANNEL,N_ACT):
        super().__init__()
        
        self.conv1 = nn.Conv2d(IN_CHANNEL,32,8,stride=4)
        self.conv2 = nn.Conv2d(32,64,4,stride=2)
        self.conv3 = nn.Conv2d(64,64,3,stride=1)
        
        self.value_fc1 = nn.Linear(7*7*64,512)
        self.value_fc2 = nn.Linear(512,1)
        
        self.advantage_fc1 = nn.Linear(7*7*64,512)
        self.advantage_fc2 = nn.Linear(512,N_ACT)
        
        #self.qvalue = nn.Linear(N_ACT)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        x = x.reshape(x.shape[0],-1)
        x1 = F.relu(self.value_fc1(x))
        x2 = F.relu(self.advantage_fc1(x))
        
        x1 = self.value_fc2(x1)
        x2 = self.advantage_fc2(x2)
        
        # expand and repeat https://zhuanlan.zhihu.com/p/58109107
        
        x = x2-x2.mean(dim=1,keepdim=True)+x1
        return x


### DQN

In [6]:
class DQN(nn.Module):
    def __init__(self,IN_CHANNEL,N_ACT):
        super().__init__()
        
        self.conv1 = nn.Conv2d(IN_CHANNEL,32,8,stride=4)
        self.conv2 = nn.Conv2d(32,64,4,stride=2)
        self.conv3 = nn.Conv2d(64,64,3,stride=1)
        
        self.fc1 = nn.Linear(7*7*64,512)
        self.fc2 = nn.Linear(512,N_ACT)
        
        #self.qvalue = nn.Linear(N_ACT)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        x = x.reshape(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Priorited Replay

In [7]:
class PReplay:
    # priorited replay buffer
    def __init__(self, MEMORY_SIZE = 5000, \
                       ALPHA = 0.5, \
                       BETA = 0.5,  \
                       BASE = 0.1,  \
                       BATCH_SIZE = 64):
        
        self.BATCH_SIZE  = BATCH_SIZE
        self.MEMORY_SIZE = MEMORY_SIZE
        self.ALPHA = ALPHA
        self.BETA  = BETA
        self.BASE  = BASE
        
        self.state_memo      = deque([],maxlen = MEMORY_SIZE)
        self.state_next_memo = deque([],maxlen = MEMORY_SIZE)
        self.act_memo    = deque([],maxlen = MEMORY_SIZE)
        self.reward_memo = deque([],maxlen = MEMORY_SIZE)
        self.done_memo   = deque([],maxlen = MEMORY_SIZE)
        
        self.priority  = deque([],maxlen = MEMORY_SIZE)
        self.prob      = deque([],maxlen = MEMORY_SIZE)
        
    def memo_append(self, ob, act,reward, ob_next, done):
        if len(self.priority) == 0:
            self.priority.append(1)
        else:            
            self.priority.append(max(self.priority))
        # a_set_memory = sars(a) : [ob, (act), reward, ob_next, done]
        self.state_memo.append(ob)
        self.state_next_memo.append(ob_next)        
        self.act_memo.append(act)
        self.reward_memo.append(reward)
        self.done_memo.append(done)
        
    def memo_len(self):
        return len(self.state_memo)
        
    def prob_update(self):
        priority_alpha = np.power(np.array(self.priority),self.ALPHA)
        self.prob = priority_alpha/np.sum(priority_alpha)
        
    def priority_update(self, error, batch_index):
        priority_array = np.array(self.priority).astype('float') # somehow turn into int64
        priority_array[batch_index] = np.abs(error) + self.BASE
        self.priority = deque(priority_array.tolist(),maxlen = self.MEMORY_SIZE)
        
    def sample(self):
        
        self.prob_update()
        batch_index = np.random.choice(range(self.memo_len()),self.BATCH_SIZE,p=self.prob,replace=False)
        
        batch_state      = np.array(self.state_memo)[batch_index]
        batch_state_next = np.array(self.state_next_memo)[batch_index]
        batch_act        = np.array(self.act_memo)[batch_index].astype(int)
        batch_reward     = np.array(self.reward_memo)[batch_index]
        batch_done       = np.array(self.done_memo)[batch_index]
        
        return batch_index,batch_state, batch_act, batch_reward, batch_state_next, batch_done

### Replay

In [8]:
class Replay:
    def __init__(self,MEMORY_SIZE = 5000, BATCH_SIZE = 32):
        self.BATCH_SIZE  = BATCH_SIZE
        self.MEMORY_SIZE = MEMORY_SIZE
        
        self.state_memo      = deque([],maxlen = MEMORY_SIZE)
        self.state_next_memo = deque([],maxlen = MEMORY_SIZE)
        self.act_memo    = deque([],maxlen = MEMORY_SIZE)
        self.reward_memo = deque([],maxlen = MEMORY_SIZE)
        self.done_memo   = deque([],maxlen = MEMORY_SIZE)
        
    def memo_append(self, ob, act,reward, ob_next, done):
        # a_set_memory = sars(a) : [ob, (act), reward, ob_next, done]
        # ob modify for dm_wrapper                                    
        self.state_memo.append(ob)
        self.state_next_memo.append(ob_next)        
        self.act_memo.append(act)
        self.reward_memo.append(reward)
        self.done_memo.append(done)
        
    def memo_len(self):
        return len(self.state_memo)
        
    
    def sample(self):
        batch_index = random.sample(range(self.memo_len()),self.BATCH_SIZE)
        
        batch_state      = np.array(self.state_memo)[batch_index]
        batch_state_next = np.array(self.state_next_memo)[batch_index]
        batch_act        = np.array(self.act_memo)[batch_index].astype(int)
        batch_reward     = np.array(self.reward_memo)[batch_index]
        batch_done       = np.array(self.done_memo)[batch_index]
        
        return batch_index, batch_state, batch_act, batch_reward, batch_state_next, batch_done

### Log

In [9]:
class Logfile:
    def __init__(self, DIR):
        self.DIR = DIR
        self.log_file = open(DIR,'a')    
        
    def open(self):
        self.log_file = open(self.DIR,'a')
        
    def write(self,string):
        print(string,end="")        
        self.log_file.write(string)
        
    def close(self):
        self.log_file.close()

## Main

In [10]:
# %%
import os
import imageio
import datetime
import random

import matplotlib.pyplot as plt
import numpy as np

from collections import deque

from torch.optim import optimizer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [11]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Running on GPU")
else:
    DEVICE = torch.device('cpu')
    print("Running on CPU")

Running on GPU


In [12]:
env = make_env(ENV_NAME)
N_ACT = env.action_space.n
N_OB  = env.observation_space.shape # 84.84.4


#FILENAME = os.path.splitext(os.path.basename(__file__))[0]
# FILENAME = "dqn"
# ROOT = f"../test_{FILENAME}_{ENV_NAME}"
# DIR = os.path.join(ROOT, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
# try:
#     os.makedirs(DIR)
# except:
#     print(f"Failed to open folder {DIR}")

# DIR_GIF = os.path.join(DIR,'GIF')
# try:
#     os.makedirs(DIR_GIF)
# except:
#     print(f"Failed to open folder {DIR_GIF}")
    

# log_file = Logfile(DIR+'/log.txt')
print(f"FRAME_END:{FRAME_END}\n"   
      f"EPSILON:{EPSILON}, EPSILON_END:{EPSILON_END},EPSILON_DECAY:{EPSILON_DECAY}\n"   
      f"GAMMA: {GAMMA},  LEARNING_RATE:{LEARNING_RATE} \n"  
      f"MEMORY_SIZE:{MEMORY_SIZE} ,MEMORY_SAMPLE_START:{MEMORY_SAMPLE_START}\n" 
      f"MODEL_UPDATE_STEP:{MODEL_UPDATE_STEP}\n"  
      f"BATCH_SIZE:{BATCH_SIZE}\n")


FRAME_END:300000.0
EPSILON:1.0, EPSILON_END:0.02,EPSILON_DECAY:0.99998
GAMMA: 0.99,  LEARNING_RATE:0.0001 
MEMORY_SIZE:10000 ,MEMORY_SAMPLE_START:1000
MODEL_UPDATE_STEP:1000
BATCH_SIZE:32



In [ ]:
ReplayBuffer = Replay(MEMORY_SIZE = MEMORY_SIZE,BATCH_SIZE = BATCH_SIZE)

model = DQN(N_OB[2],N_ACT).to(DEVICE)
target_model = DQN(N_OB[2],N_ACT).to(DEVICE)
target_model.load_state_dict(model.state_dict())
        
optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)
loss_func = nn.MSELoss()
  
    
def get_action(state): # get action with epsilon greedy
    # state modify for dm_wrapper
    with torch.no_grad():
        q = model(state)
    if np.random.rand() <= EPSILON:            
        return np.random.randint(N_ACT)
    return torch.argmax(q)

    
def replay_sample_tensor():   
    batch_index, batch_state, batch_act, batch_reward, batch_state_next, batch_done = ReplayBuffer.sample()

    batch_state      = torch.Tensor(batch_state).permute(0,3,1,2).to(DEVICE)
    batch_state_next = torch.Tensor(batch_state_next).permute(0,3,1,2).to(DEVICE)
    
    
    batch_act    = torch.Tensor(batch_act).type(torch.LongTensor).to(DEVICE)
    batch_reward = torch.Tensor(batch_reward).to(DEVICE)
    batch_done   = torch.Tensor(batch_done).to(DEVICE)
    return batch_index,batch_state, batch_act, batch_reward, batch_state_next, batch_done

# %%               
reward_sum = []
frame_sum = 0
ep = 0

TEST = False
# %%
while(1):

    reward_list = 0
    step = 0
    if GIF_MAKE: images = []
    ob = env.reset()
    
    # print(model.fc2.bias)
    # print(target_model.fc2.bias)
    #======================================================
    while(1):
        if TEST:
            EPSILON = 0
        else:
            EPSILON = max([EPSILON_END,EPSILON*EPSILON_DECAY])
            
        if RENDER:
            env.render()
        if GIF_MAKE:
            images.append(env.render(mode='rgb_array'))

        state = torch.Tensor(ob.concatenate()).view(-1,N_OB[2],N_OB[0],N_OB[1]).to(DEVICE)
        act = get_action(state)
        ob_next, reward, done, info = env.step(act)

        #ob_next_array = ob_next.concatenate()        
        ReplayBuffer.memo_append(ob.concatenate(),act,reward,ob_next.concatenate(),done)
        
        # batch train==============================
        if ReplayBuffer.memo_len() >= MEMORY_SAMPLE_START:
            batch_index,batch_state, batch_act, batch_reward, batch_state_next, batch_done = replay_sample_tensor()
            
            range_index = torch.arange(0,BATCH_SIZE,device=DEVICE)
            batch_act = batch_act.type(torch.LongTensor).to(DEVICE)
            predict_q = model(batch_state)[range_index,batch_act] # [32,4] batch_q = model(batch_state)
            
            # target_model for max q
            with torch.no_grad():
                target_q = batch_reward + \
                            (1-batch_done)*GAMMA*torch.amax(target_model(batch_state_next),dim=1)
            # pytorch sgd
            del batch_state, batch_act, batch_reward, batch_state_next, batch_done 

            #ReplayBuffer.priority_update((target_q-predict_q).cpu().detach().numpy(),batch_index)
            optimizer.zero_grad()
            loss = loss_func(predict_q,target_q)    
            #loss = loss.clamp(-1,1)
            loss.backward()
            optimizer.step()                

        # copy parameters==============================
        if  frame_sum % MODEL_UPDATE_STEP==0:
            # print("\nModel Bias:",model.fc2.bias.data)
            target_model.load_state_dict(model.state_dict()) 
        
        ob = ob_next
        step += 1
        frame_sum +=1
        reward_list += reward
            
        if done:            
            break
    #==================================    
    ep += 1
    
    reward_sum.append(reward_list)
    
    print(f"\nEpoch: {ep} \tstep: {step}" 
          f"\tframe_sum: {frame_sum}"
          f"\tepsilon: {EPSILON:.3f}"
          f"\tmean_rewards: {np.mean(reward_sum[-100:]):.3f}"
          f"\tsum_rewards: {reward_sum[-1]:.3f}",end="")

    
    if GIF_MAKE and reward_sum[-1]: # last reward > 0
        gifname=f"final_r_{reward_sum[-1]}.gif" if TEST else f"{ep}_r_{reward_sum[-1]}.gif"
        imageio.mimsave(os.path.join(DIR_GIF,gifname),images,fps=40)

    if TEST:        
        break
    
    if frame_sum>FRAME_END or reward_sum[-1]>20:
        TEST = True
        GIF_MAKE = True
    
        # train end, rund
#=============== Show the reward every ep
print('Show the reward every ep')
plt.figure()
plt.plot(reward_sum)
plt.savefig(DIR + '/rewards.png')

torch.save(model.state_dict(),os.path.join(DIR,'Model.pt'))
print("Done")


Epoch: 1 	step: 822	frame_sum: 822	epsilon: 0.984	mean_rewards: -21.000	sum_rewards: -21.000
Epoch: 2 	step: 850	frame_sum: 1672	epsilon: 0.967	mean_rewards: -21.000	sum_rewards: -21.000
Epoch: 3 	step: 1167	frame_sum: 2839	epsilon: 0.945	mean_rewards: -19.667	sum_rewards: -17.000
Epoch: 4 	step: 820	frame_sum: 3659	epsilon: 0.929	mean_rewards: -20.000	sum_rewards: -21.000
Epoch: 5 	step: 871	frame_sum: 4530	epsilon: 0.913	mean_rewards: -20.200	sum_rewards: -21.000
Epoch: 6 	step: 773	frame_sum: 5303	epsilon: 0.899	mean_rewards: -20.333	sum_rewards: -21.000
Epoch: 7 	step: 755	frame_sum: 6058	epsilon: 0.886	mean_rewards: -20.429	sum_rewards: -21.000
Epoch: 8 	step: 927	frame_sum: 6985	epsilon: 0.870	mean_rewards: -20.375	sum_rewards: -20.000
Epoch: 9 	step: 977	frame_sum: 7962	epsilon: 0.853	mean_rewards: -20.333	sum_rewards: -20.000
Epoch: 10 	step: 789	frame_sum: 8751	epsilon: 0.839	mean_rewards: -20.400	sum_rewards: -21.000
Epoch: 11 	step: 843	frame_sum: 9594	epsilon: 0.825	mean_r